In [88]:
# python version utilised Python 3.10.6

%pip install plotly==5.14.1 -q
%pip install jupyter-dash==0.4.2 -q
%pip install yfinance==0.2.18 -q
%pip install numpy==1.21.5
%pip install numpy==1.21.5
%pip install scipy==1.9.3

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


The code "pip show package_name" is used to display information about the package installed in the Python environment. This command shows the metadata associated with the package, including the version number, location, and dependencies. The output of this command includes the name, version, location, summary, homepage, author, license, and dependencies of the package. 

In [89]:
import pandas
import pandas as pd

import numpy
import numpy as np

from datetime import datetime

In [90]:
def convert_column_names_to_snake_case(data : pandas.core.frame.DataFrame) -> pandas.core.frame.DataFrame:
    """
     Converts column names to snake case. This is useful for converting a DataFrame to a DataFrame that can be used in a SonarQube command
     
     @param data - Dataframe to be converted to snake case
     
     @return Dataframe with columns converted to snake case and renamed
    """
    data.columns = data.columns.str.lower()
    to_snake_case = lambda x: x.lower().replace(' ', '_')
    return data.rename(columns=to_snake_case)


def convert_to_string_date(input :str) ->str:
    # input is a string, of format '2022-04-20'
    return datetime.strptime(input, '%Y-%m-%d').strftime('%d-%B-%Y')

# Download the dataset from Yahoo Finance. If you prefer, you can choose another source.

In [91]:
import yfinance as yf
from datetime import datetime, timedelta

ticker = yf.Ticker("AAPL")
company_name = "Apple Inc"

# end_date = is today's date
end_date = datetime.today().strftime('%Y-%m-%d')

# start_date = start date which is one year before today's date (today's date - 1 year)
start_date = (datetime.today() - timedelta(days=365)).strftime('%Y-%m-%d')

period = str(convert_to_string_date(start_date) + " to " + convert_to_string_date(end_date))

# Download data for the past 1 year
data = yf.download("AAPL", start=start_date, end=end_date)

# Rename columns to snakecase, Renaming all the column values to snakecase it can make the column names more consistent and easier to work with.
data = convert_column_names_to_snake_case(data)
display(data)

[*********************100%***********************]  1 of 1 completed


,open,high,low,close,adj_close,volume
Date,,,,,,
2022-04-20,168.759995,168.880005,166.100006,167.229996,166.223053,67929800
2022-04-21,168.910004,171.529999,165.910004,166.419998,165.417938,87227800
2022-04-22,166.460007,167.869995,161.500000,161.789993,160.815811,84882400
2022-04-25,161.119995,163.169998,158.460007,162.880005,161.899261,96046400
2022-04-26,162.250000,162.339996,156.720001,156.800003,155.855881,95623200
...,...,...,...,...,...,...
2023-04-13,161.630005,165.800003,161.419998,165.559998,165.559998,68445600
2023-04-14,164.589996,166.320007,163.820007,165.210007,165.210007,49337200
2023-04-17,165.089996,165.389999,164.029999,165.229996,165.229996,41516200


# Normal returns
Normal return is a measure of the percentage change in the price of an asset over a period of time, expressed as a percentage of its initial price. It is calculated by taking the difference between the final price and the initial price, dividing by the initial price, and multiplying by 100. Normal return is a simple and easy-to-understand measure of investment performance, but it does not take into account the time value of money or the compounding effect of returns.

# Log returns
Log return is a measure of the percentage change in the logarithm of the price of an asset over a period of time. It is calculated by taking the natural logarithm of the final price divided by the initial price. Log return is a more accurate measure of investment performance than normal return because it takes into account the time value of money and the compounding effect of returns. It is also useful for modeling financial data because it has several desirable statistical properties, such as being normally distributed and having stationary increments.

In [92]:

returns = pd.DataFrame()

# calculate normal returns
returns['normal_return'] = data['close'].pct_change()

# calculate log returns
returns['log_return'] = np.log(data['close']) - np.log(data['close'].shift(1))

# display the new DataFrame
display(returns.head(5))

# drop NaN values
returns = returns.dropna()

# display the new DataFrame after dropping NaN values
display(returns.head(5))

,normal_return,log_return
Date,,
2022-04-20,NaN,NaN
2022-04-21,-0.004844,-0.004855
2022-04-22,-0.027821,-0.028216
2022-04-25,0.006737,0.006715
2022-04-26,-0.037328,-0.038043


,normal_return,log_return
Date,,
2022-04-21,-0.004844,-0.004855
2022-04-22,-0.027821,-0.028216
2022-04-25,0.006737,0.006715
2022-04-26,-0.037328,-0.038043
2022-04-27,-0.001467,-0.001468


In [93]:
import plotly.graph_objects as go

def plot_linegraph( x_col, y_col, title, x_label, y_label):
    """
     Plot a line graph. This is a wrapper around go. Scatter to make it easier to visualize the plot
     
     @param x_col - column of x data ( int )
     @param y_col - column of y data ( int )
     @param title - title of the plot ( str ). Default is'Plot '
     @param x_label - x - axis label ( str )
     @param y_label - y - axis label ( str )
    """
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x_col, y=y_col, mode='lines'))
    fig.update_layout(title=title, xaxis_title=x_label, yaxis_title=y_label)
    fig.show()

In [94]:
# Plots line graph of normal returns of company. Parameters Returns Nothing but generates a plot
plot_linegraph(x_col=returns.index,y_col=returns["normal_return"] ,title="Normal Returns of "+company_name+ " during "+ period,x_label="date",y_label="noraml returns")

In [95]:
# Plots line graph of log returns of company. Parameters Returns Nothing but generates a plot
plot_linegraph(x_col=returns.index,y_col=returns["log_return"] ,title="Log Returns of "+company_name + " during "+ period,x_label="date",y_label="log returns")

# Descriptive statistics of the returns

In [96]:
from scipy import stats

def calculate_descriptive_statistics(ds):
    """
     Calculates descriptive statistics about the return values. This is used to show a list of statistics that are relevant to the company
     
     @param ds - the data set returned
    """
    
    print(f"Number of observations is {ds.nobs:.0f}")
    print(f"Minimum return of {company_name} during the {period}: {(ds.minmax[0] * 100):.3f} %")
    print(f"Maximum return of {company_name} during the {period}: {(ds.minmax[1] * 100):.3f} %")
    print(f"Average return of {company_name} during the {period}: {(ds.mean)} %")
    
    # The variance of returns measures the dispersion of the returns around their mean value. A high variance indicates that the returns are more spread out and the asset or portfolio is more volatile, while a low variance indicates that the returns are more tightly clustered around the mean and the asset or portfolio is less volatile.
    print(f"Variance of {company_name} during the period: {(ds.variance):.7f}")

    # Prints the skewness of stock returns.
    if ds.skewness >= 0:
        print(f"Skewness : {ds.skewness}. If the skewness of stock returns is positive, it means that the distribution of returns is skewed to the right, with more returns on the positive side of the mean and fewer returns on the negative side.")
    else:
        print(f"Skewness : {ds.skewness}. If the skewness of stock returns is negative, it means that the distribution of returns is skewed to the left, with more returns on the negative side of the mean and fewer returns on the positive side.")

    # Prints a list of kurtosis values of zero or less than zero.
    if ds.kurtosis > 0:
        print(f"Kurtosis : {ds.kurtosis}. Positive kurtosis, also known as leptokurtosis, indicates that the distribution of returns has more extreme values than a normal distribution. This suggests that the stock returns have a higher probability of experiencing larger deviations from the mean, or more frequent large positive or negative returns. This could be an indication of high volatility, higher risk, or potential for large gains or losses.")
    elif ds.kurtosis == 0:
        print(f"Kurtosis : {ds.kurtosis}. A kurtosis value of zero indicates that the distribution is similar to a normal distribution in terms of the presence of outliers.")
    else:
        print(f"Kurtosis : {ds.kurtosis}. Negative kurtosis, also known as platykurtosis, indicates that the distribution of returns has fewer extreme values than a normal distribution. This suggests that the stock returns have a lower probability of experiencing large deviations from the mean, or less frequent large positive or negative returns. This could be an indication of lower volatility, lower risk, or less potential for large gains or losses.")


In [97]:
# Discriptive stats of returns 

normal_return_ds = stats.describe(returns['normal_return'])
print("Discriptive stats of noramal returns\n")
calculate_descriptive_statistics(normal_return_ds)
print("\n")

print("Discriptive stats of log returns\n")
log_return_ds = stats.describe(returns['log_return'])
calculate_descriptive_statistics(log_return_ds)
print("\n")

Discriptive stats of noramal returns

Number of observations is 250
Minimum return of Apple Inc during the 20-April-2022 to 20-April-2023: -5.868 %
Maximum return of Apple Inc during the 20-April-2022 to 20-April-2023: 8.897 %
Average return of Apple Inc during the 20-April-2022 to 20-April-2023: 0.00024493761391054705 %
Variance of Apple Inc during the period: 0.0004732
Skewness : 0.11596899208875504. If the skewness of stock returns is positive, it means that the distribution of returns is skewed to the right, with more returns on the positive side of the mean and fewer returns on the negative side.
Kurtosis : 1.1171755453182666. Positive kurtosis, also known as leptokurtosis, indicates that the distribution of returns has more extreme values than a normal distribution. This suggests that the stock returns have a higher probability of experiencing larger deviations from the mean, or more frequent large positive or negative returns. This could be an indication of high volatility, high

In [98]:
from scipy.stats import shapiro

def shapiro_test(data, alpha=0.05):
    n = len(data)
    if (not n < 50) or (not n > 2000):
        print(f"Warning: Shapiro-Wilk test may not be appropriate for sample size of {n}. Consider alternative methods for assessing normality.")
    
    # Set up hypotheses
    print("H0: Sample comes from a normal distribution")
    print("H1: Sample does not come from a normal distribution")
    print(f"Significance level: {alpha}")
    
    # Perform test and print results
    stat, p = shapiro(data)
    print('Shapiro-Wilk test statistic:', stat)
    print('Shapiro-Wilk test p-value:', p)
    
    if p > alpha:
        print(f"p-value {p} > alpha {alpha}. Data looks Gaussian (fail to reject H0), sample comes from a normal distribution")
    else:
        print(f"p-value {p} < alpha {alpha}. Data does not look Gaussian (reject H0), sample does not come from a normal distribution")


In [99]:
shapiro_test(returns['log_return'])

H0: Sample comes from a normal distribution
H1: Sample does not come from a normal distribution
Significance level: 0.05
Shapiro-Wilk test statistic: 0.9865536093711853
Shapiro-Wilk test p-value: 0.01903645135462284
p-value 0.01903645135462284 < alpha 0.05. Data does not look Gaussian (reject H0), sample does not come from a normal distribution


In [100]:
shapiro_test(returns['normal_return'])

H0: Sample comes from a normal distribution
H1: Sample does not come from a normal distribution
Significance level: 0.05
Shapiro-Wilk test statistic: 0.9856709241867065
Shapiro-Wilk test p-value: 0.013141745701432228
p-value 0.013141745701432228 < alpha 0.05. Data does not look Gaussian (reject H0), sample does not come from a normal distribution


In [101]:
import plotly.express as px
import plotly.graph_objects as go
import numpy as np
from scipy.stats import gaussian_kde

def create_histogram_density(data, x_col):
    """
    Create a histogram with a density curve of the data and display it.
    
    Parameters:
        data (pandas.DataFrame): The data to plot.
        x_col (str): The column name to plot on the x-axis.
        
    Returns:
        None
    """
    
    # Create the histogram
    fig = px.histogram(data, x=x_col, histnorm='probability density')

    # Compute the density curve
    density = gaussian_kde(data[x_col])
    x = np.linspace(data[x_col].min(), data[x_col].max(), 100)
    y = density(x)

    # Add the density curve to the histogram
    fig.add_trace(go.Scatter(x=x, y=y, mode='lines', name='Density'))
    
    # Add a title to the figure
    fig.update_layout(title='Histogram with Density Curve of ' + x_col)

    # Display the figure
    fig.show()


In [102]:
create_histogram_density(returns, 'normal_return')

In [103]:
create_histogram_density(returns, 'log_return')

In [104]:
from scipy.stats import anderson
import numpy as np

def anderson_noramlity_test(data: pandas.core.series.Series):
    
    print("Anderson-Darling test: There is no fixed sample size for the Anderson-Darling test. The test can be used for any sample size, but it is more powerful for larger sample sizes.")
    print("H0 is that the sample is drawn from a population that follows a particular distribution, such as a normal distribution")
    print("H1 is that the sample is not drawn from the specified distribution")

    # perform the Anderson-Darling test
    result = anderson(data)

    # print the test statistic and critical values
    print('Statistic: %.3f' % result.statistic)
    for i in range(len(result.critical_values)):
        sl, cv = result.significance_level[i], result.critical_values[i]
        if result.statistic < cv:
            print('%.3f: %.3f, data looks normal (fail to reject H0)' % (sl, cv))
        else:
            print('%.3f: %.3f, data does not look normal (reject H0)' % (sl, cv))

In [105]:
anderson_noramlity_test(returns['normal_return'])

Anderson-Darling test: There is no fixed sample size for the Anderson-Darling test. The test can be used for any sample size, but it is more powerful for larger sample sizes.
H0 is that the sample is drawn from a population that follows a particular distribution, such as a normal distribution
H1 is that the sample is not drawn from the specified distribution
Statistic: 0.613
15.000: 0.567, data does not look normal (reject H0)
10.000: 0.646, data looks normal (fail to reject H0)
5.000: 0.775, data looks normal (fail to reject H0)
2.500: 0.904, data looks normal (fail to reject H0)
1.000: 1.075, data looks normal (fail to reject H0)


In [106]:
anderson_noramlity_test(returns['log_return'])

Anderson-Darling test: There is no fixed sample size for the Anderson-Darling test. The test can be used for any sample size, but it is more powerful for larger sample sizes.
H0 is that the sample is drawn from a population that follows a particular distribution, such as a normal distribution
H1 is that the sample is not drawn from the specified distribution
Statistic: 0.683
15.000: 0.567, data does not look normal (reject H0)
10.000: 0.646, data does not look normal (reject H0)
5.000: 0.775, data looks normal (fail to reject H0)
2.500: 0.904, data looks normal (fail to reject H0)
1.000: 1.075, data looks normal (fail to reject H0)


In [107]:
import numpy as np
from scipy.stats import kurtosistest

def kurtosis_noramlity_test(data:  pandas.core.series.Series): 
    print("kurtosis test")
    print("H0: The sample comes from a normal distribution (i.e., the population has a kurtosis value of 3)")
    print("H1: The sample does not come from a normal distribution (i.e., the population does not have a kurtosis value of 3)")

    # Perform the kurtosis test
    statistic, pvalue = kurtosistest(data)
    print(f"statistic: {statistic}")
    print(f"p-value: {pvalue}")

    # Set the significance level
    alpha = 0.05

    # Interpret the results
    if pvalue < alpha:
        print("Reject the null hypothesis. The sample does not have a normal distribution.")
    else:
        print("Fail to reject the null hypothesis. The sample may have a normal distribution.")


In [108]:
kurtosis_noramlity_test(returns['normal_return'])

kurtosis test
H0: The sample comes from a normal distribution (i.e., the population has a kurtosis value of 3)
H1: The sample does not come from a normal distribution (i.e., the population does not have a kurtosis value of 3)
statistic: 2.7457827914046855
p-value: 0.006036671615949909
Reject the null hypothesis. The sample does not have a normal distribution.


In [109]:
kurtosis_noramlity_test(returns['log_return'])

kurtosis test
H0: The sample comes from a normal distribution (i.e., the population has a kurtosis value of 3)
H1: The sample does not come from a normal distribution (i.e., the population does not have a kurtosis value of 3)
statistic: 2.5153404370597277
p-value: 0.011891749284090853
Reject the null hypothesis. The sample does not have a normal distribution.


# Can price movement can be described as a random walk

In [112]:
data['close_lag_1'] = data['close'].shift(1)
data['close_lag_2'] = data['close'].shift(2)

# drop all rows containing missing values
data.dropna(inplace=True)

display(data.head(5))

,open,high,low,close,adj_close,volume,close_lag_1,close_lag_2
Date,,,,,,,,
2022-04-22,166.460007,167.869995,161.500000,161.789993,160.815811,84882400,166.419998,167.229996
2022-04-25,161.119995,163.169998,158.460007,162.880005,161.899261,96046400,161.789993,166.419998
2022-04-26,162.250000,162.339996,156.720001,156.800003,155.855881,95623200,162.880005,161.789993
2022-04-27,155.910004,159.789993,155.380005,156.570007,155.627274,88063200,156.800003,162.880005
2022-04-28,159.250000,164.520004,158.929993,163.639999,162.654678,130216800,156.570007,156.800003


# Fit linear model

In [114]:
coefficients = np.linalg.lstsq(data[['close_lag_1', 'close_lag_2']], data['close'], rcond=None)[0]
print(coefficients)

[0.9314376  0.06838168]


In [117]:
data['predicted_close'] = np.dot(data[['close_lag_1', 'close_lag_2']],coefficients)
display(data.head(10))

,open,high,low,close,adj_close,volume,close_lag_1,close_lag_2,predicted_clos,predicted_close
Date,,,,,,,,,,
2022-04-22,166.460007,167.869995,161.500000,161.789993,160.815811,84882400,166.419998,167.229996,166.445311,166.445311
2022-04-25,161.119995,163.169998,158.460007,162.880005,161.899261,96046400,161.789993,166.419998,162.077361,162.077361
2022-04-26,162.250000,162.339996,156.720001,156.800003,155.855881,95623200,162.880005,161.789993,162.776032,162.776032
2022-04-27,155.910004,159.789993,155.380005,156.570007,155.627274,88063200,156.800003,162.880005,157.187426,157.187426
2022-04-28,159.250000,164.520004,158.929993,163.639999,162.654678,130216800,156.570007,156.800003,156.557439,156.557439
2022-04-29,161.839996,166.199997,157.250000,157.649994,156.700745,131747600,163.639999,156.570007,163.126968,163.126968
2022-05-02,156.710007,158.229996,153.270004,157.960007,157.008881,123055300,157.649994,163.639999,158.031109,158.031109
2022-05-03,158.149994,160.710007,156.320007,159.479996,158.519730,88966500,157.960007,157.649994,157.910260,157.910260
2022-05-04,159.669998,166.479996,159.259995,166.020004,165.020370,108256500,159.479996,157.960007,159.347234,159.347234


In [118]:
import plotly.graph_objs as go
import pandas as pd

# Create a Plotly figure object
fig = go.Figure()

# Add a trace for the predicted close values
fig.add_trace(go.Scatter(x=data.index, y=data['predicted_close'], name='Predicted Close'))

# Add a trace for the actual close values
fig.add_trace(go.Scatter(x=data.index, y=data['close'], name='Actual Close'))

# Add a title and axis labels
fig.update_layout(title='Predicted vs Actual Close Prices', xaxis_title='Date', yaxis_title='Price')

# Show the plot
fig.show()
